## This notebook contains a bunch of scrape code pieces that were useful every now and then.

In [175]:
# File location
floc = '/home/dante/SpatialData/spatial_project/data/google/2020_FI_Region_Mobility_Report.csv'

# Assigning dtypes to the to_csv function, as apparently it is very computationally hard
# for Pandas to try and guess dtypes automatically.
dtypes = {
    'country_region_code' : 'object',
    'country_region' : 'object',
    'sub_region_1' : 'object',
    'sub_region_2' : 'object',
    'metro_area' : 'object',
    'iso_3166_2_code' : 'object',
    'census_fips_code' : 'object',
    'place_id' : 'object',
    'date' : 'object',
    'retail_and_recreation_percent_change_from_baseline' : 'float64',
    'grocery_and_pharmacy_percent_change_from_baseline' : 'float64',
    'parks_percent_change_from_baseline' : 'float64',
    'transit_stations_percent_change_from_baseline' : 'float64',
    'workplaces_percent_change_from_baseline' : 'float64',
    'residential_percent_change_from_baseline' : 'float64',
}

# Assigning the correct column names
headers = list(dtypes.keys())

# Telling Pandas what column is a datecolumn (so that it can be parsed)
parse_dates = ['date']

# Reading in file
df_mr = pd.read_csv(floc,header=0,dtype=dtypes,names=headers,parse_dates=parse_dates)

# Sorting values by date and sub_region_2,sub_region_1 as we want to be able to
# look at the data in an intuitive way.
df_mr.sort_values(by=['date','sub_region_2','sub_region_1'],inplace=True)

In [176]:
# Creating a map between the ISO-codes of the different provinces and their ENG/FI/SWE 
# names, for legibility later on.
df_codemap = pd.read_csv('/home/dante/SpatialData/spatial_project/data/google/mob_map.txt',sep=',').iloc[:,0:4]
iso_name_map = dict(zip(df_codemap.iloc[:,0],df_codemap.iloc[:,3]))
df_mr['province'] = df_mr.iso_3166_2_code.map(iso_name_map)


# Dropping unneccesary columns
df_mr.drop(columns=['country_region_code','country_region','census_fips_code','iso_3166_2_code','place_id','metro_area','sub_region_2','sub_region_1'],inplace=True)

# Specifying down our data to the province level
df_mr.dropna(subset=['province'],inplace=True)
df_mr.reset_index(drop=True,inplace=True)

# Casting date in correct format (apparently Pandas doesn't actually read the dtypes correctly)
df_mr['date'] = pd.to_datetime(df_mr.date)

# Adding day of year for gap checking if needed
df_mr['dayofyear'] = df_mr['date'].dt.dayofyear

# Defining the movement data columns of interest
# and filling in NaNs with 0 (i.e no change/information).
# I do not know how sustainable this assumption is...
mvcols = [col for col in df_mr.columns if 'baseline' in col]
values = {col : 0 for col in mvcols}
df_mr.fillna(value=values, inplace=True)

In [177]:
# Checking out the data
df_mr.head(10)

,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,province,dayofyear
0,2020-02-15,1.0,2.0,4.0,9.0,-5.0,0.0,Central Finland,46
1,2020-02-15,9.0,22.0,0.0,0.0,-2.0,0.0,Central Ostrobothnia,46
2,2020-02-15,6.0,14.0,17.0,0.0,3.0,0.0,Kainuu,46
3,2020-02-15,5.0,2.0,-17.0,6.0,1.0,0.0,Kymenlaakso,46
4,2020-02-15,17.0,21.0,39.0,56.0,2.0,-1.0,Lapland,46
...,...,...,...,...,...,...,...,...,...
5755,2020-12-31,-10.0,28.0,0.0,-15.0,-59.0,12.0,Southern Ostrobothnia,366
5756,2020-12-31,-16.0,25.0,119.0,-46.0,-51.0,11.0,Southern Savonia,366
5757,2020-12-31,-17.0,15.0,90.0,-38.0,-60.0,13.0,Southwest Finland,366
5758,2020-12-31,-5.0,32.0,0.0,-35.0,-60.0,13.0,Tavastia Proper,366


In [178]:
# Printing the names of the provinces
iso_name_map

{'FI-01': 'Åland',
 'FI-02': 'South Karelia',
 'FI-03': 'Southern Ostrobothnia',
 'FI-04': 'Southern Savonia',
 'FI-05': 'Kainuu',
 'FI-06': 'Tavastia Proper',
 'FI-07': 'Central Ostrobothnia',
 'FI-08': 'Central Finland',
 'FI-09': 'Kymenlaakso',
 'FI-10': 'Lapland',
 'FI-11': 'Pirkanmaa',
 'FI-12': 'Ostrobothnia',
 'FI-13': 'North Karelia',
 'FI-14': 'Northern Ostrobothnia',
 'FI-15': 'Northern Savonia',
 'FI-16': 'Päijänne Tavastia',
 'FI-17': 'Satakunta',
 'FI-18': 'Uusimaa',
 'FI-19': 'Southwest Finland'}

In [180]:
# Printing the province data shapes to make sure our data is comparable...
for idx, prov in enumerate(list(df_mr.province.value_counts().index)):
    print(f"Province {idx} {prov} has shape {df_mr[df_mr.province == prov].shape}")

# based on the above, we will get rid of Åland, because it is not in this dataset,
# nor is it connected geographically to the mainland. Would be interesting to try
# and estimate the connectedness by numbers of ferry crossings between Åland and mainland Finland
# as well as mainland Sweden!!! Just to prove that it matters... Here, the weighting would be by
# day by how many ferry rides across!

Province 0 Central Finland has shape (321, 9)
Province 1 Ostrobothnia has shape (321, 9)
Province 2 Tavastia Proper has shape (321, 9)
Province 3 Southwest Finland has shape (321, 9)
Province 4 Southern Savonia has shape (321, 9)
Province 5 Southern Ostrobothnia has shape (321, 9)
Province 6 Satakunta has shape (321, 9)
Province 7 Päijänne Tavastia has shape (321, 9)
Province 8 Pirkanmaa has shape (321, 9)
Province 9 Northern Savonia has shape (321, 9)
Province 10 Northern Ostrobothnia has shape (321, 9)
Province 11 Lapland has shape (321, 9)
Province 12 Kymenlaakso has shape (321, 9)
Province 13 Uusimaa has shape (321, 9)
Province 14 South Karelia has shape (318, 9)
Province 15 North Karelia has shape (318, 9)
Province 16 Central Ostrobothnia has shape (315, 9)
Province 17 Kainuu has shape (315, 9)


### Creating loop that will make the dataframe just like we want to i.e. in right format:
1) Year-Week Province level

In [183]:
# Creating empty list, so that we can then collect all separate 
# province dataframes into this list when processed by the below
# for - loop...
df_list = []

In [184]:
# This for - loop puts the data in the week - province format.
# Notice that we are using the MEAN movement per week!

# For province in provinces...
for prov in iso_name_map.values():
    
    # Specify the correct province 
    df = df_mr[df_mr.province == prov]
    
    # The below code bins the data according to week commencing MONDAY
    # This code looks complicated, but achieves something quite understanable.
    df['date'] = pd.to_datetime(df['date']) - pd.to_timedelta(7, unit='d')
    df = df.groupby([pd.Grouper(key='date', freq='W-MON')])[mvcols].mean().reset_index().sort_values('date')

    # Creating the column with which we can then merge to the cases data..
    df['Y-W'] = df.date.dt.year.astype(str)+'-'+df.date.dt.week.astype(str)
    
    # Adding name of province
    df['province'] = prov
    df.reset_index(drop=True) 
    
    # Finally, appending to the list of dfs...
    df_list.append(df)

/tmp/ipykernel_20002/2885164148.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['Y-W'] = df.date.dt.year.astype(str)+'-'+df.date.dt.week.astype(str)
/tmp/ipykernel_20002/2885164148.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['diff']=df.dayofyear.diff()
/tmp/ipykernel_20002/2885164148.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date']) - pd.to_timedelta(7, unit='d')
/tm

In [188]:
# Combining all the province df's into one big dataframe.
combined_mv_df = pd.concat(df_list)

In [189]:
# Having a look at what the end result looks like.
combined_mv_df.head(10)

,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,Y-W,province
0,2020-02-10,4.333333,3.666667,-7.666667,2.666667,-0.666667,0.333333,2020-7,South Karelia
1,2020-02-17,4.142857,3.857143,-2.714286,3.142857,-5.428571,1.142857,2020-8,South Karelia
2,2020-02-24,-1.857143,-1.000000,7.571429,0.428571,-20.000000,1.857143,2020-9,South Karelia
3,2020-03-02,4.285714,4.000000,2.571429,-2.000000,-0.714286,1.000000,2020-10,South Karelia
4,2020-03-09,-9.000000,3.857143,-2.285714,-19.000000,-7.857143,2.142857,2020-11,South Karelia
...,...,...,...,...,...,...,...,...,...
42,2020-11-30,-15.000000,-2.571429,17.285714,-34.857143,-16.571429,8.000000,2020-49,Southwest Finland
43,2020-12-07,-13.285714,-1.714286,16.285714,-38.285714,-17.714286,8.857143,2020-50,Southwest Finland
44,2020-12-14,-8.142857,3.000000,12.428571,-40.000000,-22.714286,10.285714,2020-51,Southwest Finland
45,2020-12-21,-35.285714,-15.000000,23.714286,-51.571429,-54.571429,13.857143,2020-52,Southwest Finland


In [190]:
# Saving the file as a CSV.
combined_mv_df.to_csv('/home/dante/SpatialData/spatial_project/data/processed/google_mobility_by_week_province.csv',index=False)

In [ ]:
nuts = pd.read_csv('/home/dante/SpatialData/spatial_project/scripts/nuts_3.txt',sep=';')

nuts.iloc[11,2] = 'Kanta-Häme'
nuts.iloc[12,2] = 'Päijät-Häme'
nuts.rename(columns={'classificationItemName' : 'nuts3'},inplace=True)
mobmap = mobmap.merge(nuts,left_on = 'nuts_code', right_on='nuts3',how='left')
mobmap = mobmap.iloc[:,1:6]
mobmap.to_csv('/home/dante/SpatialData/spatial_project/scripts/mob_map_nuts3.txt')

## Exttracting finland from SCI data

In [ ]:
# dtypes = {
#     'user_loc' : 'object',
#     'fr_loc' : 'object',
#     'scaled_sci' : 'int',
# }

# # Assigning the correct column names
# headers = list(dtypes.keys())

# # # Telling Pandas what column is a datecolumn (so that it can be parsed)
# # parse_dates = ['date']

# # Creating a map between the ISO-codes of the different provinces and their ENG/FI/SWE 
# # names, for legibility later on.
# df_codemap = pd.read_csv('/home/dante/SpatialData/spatial_project/data/fb/SCI_gadm1_nuts3_October_2021.txt',sep='\t',dtype=dtypes,names=headers,header=0)
# df_sci_fi = df_codemap[df_codemap.user_loc.str.contains('FI') & df_codemap.fr_loc.str.contains('FI')]
# del df_codemap

## Creating SCI weights

In [ ]:
# df_sci_fi = pd.read_csv('/home/dante/SpatialData/spatial_project/data/fb/SCI_fin.scv')
# mobmap = pd.read_csv('/home/dante/SpatialData/spatial_project/scripts/mob_map_nuts3.txt')

# df_sci_fi = df_sci_fi[(df_sci_fi.user_loc != 'FI200') & (df_sci_fi.fr_loc != 'FI200')]

# df_list = []
# for prov in df_sci_fi.user_loc.value_counts().index:
#     df = df_sci_fi[(df_sci_fi.user_loc == prov) & (df_sci_fi.fr_loc != prov)]
   
#     sci_tot = df.scaled_sci.sum()
#     df['sci_weights'] = df.scaled_sci / sci_tot
#     df['sci_tot'] = sci_tot
    
#     print(df)
    
#     df_list.append(df)

# df_sci = pd.concat(df_list)   

# nuts_to_provname = dict(zip(mobmap.nuts_code,mobmap.iloc[:,4]))
# df_sci['user_loc_province'] = df_sci.user_loc.map(nuts_to_provname)
# df_sci['fr_loc_province'] = df_sci.fr_loc.map(nuts_to_provname)

## THL v. official namings

In [ ]:
stat_finland_naming_v_official = {
    'Central Finland' : 'Central Finland',
    'Central Ostrobothnia' : 'Central Ostrobothnia',
    'Kainuu' : 'Kainuu',
    'Kymenlaakso' : 'Kymenlaakso',
    'North Karelia' : 'North Karelia',
    'Northern Ostrobothnia' : 'North Ostrobothnia',
    'Northern Savonia' : 'North Savo',
    'Ostrobothnia' : 'Ostrobothnia',
    'Pirkanmaa' : 'Pirkanmaa',
    'Päijänne Tavastia' : 'Päijät-Häme',
    'Satakunta' : 'Satakunta',
    'South Karelia' : 'South Karelia',
    'Southern Ostrobothnia' : 'South Ostrobothnia',
    'Southern Savonia' : 'South Savo',
    'Southwest Finland' : 'Southwest Finland',
    'Tavastia Proper' : 'Kanta-Häme',
    'Uusimaa' : 'Uusimaa',
    'Lapland' : 'Lapland'
}



prov_nam_conv['thl_namings_eng'] = np.select([
                                                  (prov_nam_conv.iloc[:,3].isin(thl_namings)),
                                                  (prov_nam_conv.iloc[:,5].isin(thl_namings))
                                             ],
                                             [
                                                 (prov_nam_conv.iloc[:,3]),
                                                 (prov_nam_conv.iloc[:,5])
                                             ],
                                                default = 'not found'
                                            )

## Preparing COVID data (HOSPITAL DISTRICT)

In [ ]:
url = "https://sampo.thl.fi/pivot/prod/en/epirapo/covid19case/fact_epirapo_covid19case.json"
response = urlopen(url)

data_json = json.loads(response.read())

In [ ]:
data_dict = dict(data_json)

In [ ]:
# print(data_dict['dataset'].keys())
# print(data_dict['dataset']['class'])
# print(data_dict['dataset']['label'])
# print(data_dict['dataset']['dimension'].keys())
# print(data_dict['dataset']['value'].keys())
# print(len(list(data_dict['dataset']['value'].values())))
# print(data_dict['dataset']['dimension']['hcdmunicipality2020']['category'].keys())
# print(data_dict['dataset']['dimension']['hcdmunicipality2020']['category']['index'].items())
# print(data_dict['dataset']['dimension']['hcdmunicipality2020']['category']['label'].items())
# print(data_dict['dataset']['dimension']['dateweek20200101']['category']['label'].values())


# defining handy quantities
hcd_to_number = {v:k for k,v in data_dict['dataset']['dimension']['hcdmunicipality2020']['category']['label'].items()}

In [ ]:
# len(data_dict['dataset']['value'])
hospital_districts = dict(data_dict['dataset']['dimension']['hcdmunicipality2020']['category']['label'])
hospital_districts_no = len(hospital_districts)

casebyweek = dict(data_dict['dataset']['value'])
weeks_no = int(len(casebyweek.values()) / len(hospital_districts.values()))

weeks_no

weeks_range = data_dict['dataset']['dimension']['dateweek20200101']['category']['label']

week_list = list(weeks_range.values()) * len(hospital_districts.values())
week_list = (week_list[0:weeks_no-1]+['Total']) * len(hospital_districts.values())

hospital_districts_rep = [entry for entry in list(hospital_districts.values()) for _ in range(weeks_no)][0:len(casebyweek.values())]

In [ ]:
datas = {
    'week' : week_list,
    'hospital_district' : hospital_districts_rep,
    'new_cases' : list(casebyweek.values())
}

for key in datas.keys():
    print(len(datas[key]))

In [ ]:
# Creating dataframe object from our dictionary
df = pd.DataFrame(datas)

# Casting new cases as int
df['new_cases'] = df.new_cases.astype(int)

# Creating cumulative sum column, with 0 if the 'week' is equal to total
df['cumsum'] = df.groupby('hospital_district')['new_cases'].cumsum()
df['cumsum'] = np.where(df['week'] == 'Total',0,df['cumsum'])

# Applying the datetimer function, then transforming into datetime
df['week_n'] = df.iloc[:,0].apply(datetimer)
df.drop(columns=['week'],axis=1,inplace=True)
df.rename(columns={'week_n':'Y_W'},inplace=True)

# Adding all data we have about each hospital district
df['hospital_district_id'] = df.iloc[:,0].map(hcd_to_number)


# Splitting into weekly all-areas, totals, and hcd by week
df_aa = df[df.hospital_district == 'All areas'] \
            .reset_index(drop=True) \
            .iloc[:-1,:]

df_tot = df[df.Y_W == 'Total'] \
            .reset_index(drop=True)\
            .drop(columns=['Y_W','cumsum']) \
            .rename(columns={'new_cases':'total'})

df_hcd_week = df[(df.Y_W != 'Total') & (df.hospital_district != 'All areas')] \
                .reset_index(drop=True) 

In [ ]:
df_aa.head(115)
# df_aa.to_csv('/home/dante/SpatialData/spatial_project/data/processed/all_area_ts.csv')

In [ ]:
df_tot.head(hospital_districts_no)
# df_tot.to_csv('/home/dante/SpatialData/spatial_project/data/processed/total_by_hcd.csv')

In [ ]:
df_hcd_week.head(weeks_no-1)
# df_hcd_week.to_csv('/home/dante/SpatialData/spatial_project/data/processed/hcd_ts.csv')

## Preparing FB movement range data

In [ ]:
floc = '/home/dante/SpatialData/spatial_project/data/fb/movement-range-data-2020-03-01-2020-12-31/movement-range-data-2020-03-01--2020-12-31.txt'
dtypes = {
    'ds' : 'object',
    'country' : 'object',
    'polygon_source' : 'object',
    'polygon_id' : 'object',
    'polygon_name' : 'object',
    'all_day_bing_tiles_visited_relative_change' : 'float64',
    'all_day_ratio_single_tile_users' : 'float64',
    'baseline_name' : 'object',
    'baseline_type' : 'object',
}
headers = list(dtypes.keys())
parse_dates = ['ds']
df_mv = pd.read_csv(floc, sep='\t', header=0, dtype=dtypes, names=headers, parse_dates=parse_dates)

In [ ]:
df_mv = df_mv[df_mv.country == 'FIN']
df_mv.polygon_name.value_counts()
del df_mv

## => movement range data for Finland is TRASH!

In [76]:
# File location
floc = '/home/dante/SpatialData/spatial_project/data/google/2020_FI_Region_Mobility_Report.csv'

# Assigning dtypes to the to_csv function, as apparently it is very computationally hard
# for Pandas to try and guess dtypes automatically.
dtypes = {
    'country_region_code' : 'object',
    'country_region' : 'object',
    'sub_region_1' : 'object',
    'sub_region_2' : 'object',
    'metro_area' : 'object',
    'iso_3166_2_code' : 'object',
    'census_fips_code' : 'object',
    'place_id' : 'object',
    'date' : 'object',
    'retail_and_recreation_percent_change_from_baseline' : 'float64',
    'grocery_and_pharmacy_percent_change_from_baseline' : 'float64',
    'parks_percent_change_from_baseline' : 'float64',
    'transit_stations_percent_change_from_baseline' : 'float64',
    'workplaces_percent_change_from_baseline' : 'float64',
    'residential_percent_change_from_baseline' : 'float64',
}

# Assigning the correct column names
headers = list(dtypes.keys())

# Telling Pandas what column is a datecolumn (so that it can be parsed)
parse_dates = ['date']

# Reading in file
df_mr = pd.read_csv(floc,header=0,dtype=dtypes,names=headers,parse_dates=parse_dates)

# Sorting values by date and sub_region_2,sub_region_1 as we want to be able to
# look at the data in an intuitive way.
df_mr.sort_values(by=['date','sub_region_2','sub_region_1'],inplace=True)

In [84]:
df_mr[df_mr.iso_3166_2_code == 'FI-18'].sort_values(by = 'date',ascending=False)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
23594,FI,Finland,Uusimaa,NaN,NaN,FI-18,NaN,ChIJjxx_x8PzjUYRoPTt2ZDEppc,2020-12-31,-25.0,7.0,33.0,-53.0,-66.0,15.0
23593,FI,Finland,Uusimaa,NaN,NaN,FI-18,NaN,ChIJjxx_x8PzjUYRoPTt2ZDEppc,2020-12-30,-20.0,3.0,-26.0,-54.0,-60.0,16.0
23592,FI,Finland,Uusimaa,NaN,NaN,FI-18,NaN,ChIJjxx_x8PzjUYRoPTt2ZDEppc,2020-12-29,-26.0,-11.0,-11.0,-56.0,-61.0,18.0
23591,FI,Finland,Uusimaa,NaN,NaN,FI-18,NaN,ChIJjxx_x8PzjUYRoPTt2ZDEppc,2020-12-28,-21.0,-9.0,5.0,-54.0,-61.0,16.0
23590,FI,Finland,Uusimaa,NaN,NaN,FI-18,NaN,ChIJjxx_x8PzjUYRoPTt2ZDEppc,2020-12-27,-39.0,-21.0,-23.0,-56.0,-12.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23278,FI,Finland,Uusimaa,NaN,NaN,FI-18,NaN,ChIJjxx_x8PzjUYRoPTt2ZDEppc,2020-02-19,2.0,-4.0,10.0,-14.0,-27.0,2.0
23277,FI,Finland,Uusimaa,NaN,NaN,FI-18,NaN,ChIJjxx_x8PzjUYRoPTt2ZDEppc,2020-02-18,-2.0,-5.0,2.0,-15.0,-27.0,3.0
23276,FI,Finland,Uusimaa,NaN,NaN,FI-18,NaN,ChIJjxx_x8PzjUYRoPTt2ZDEppc,2020-02-17,1.0,-4.0,0.0,-14.0,-28.0,3.0
23275,FI,Finland,Uusimaa,NaN,NaN,FI-18,NaN,ChIJjxx_x8PzjUYRoPTt2ZDEppc,2020-02-16,-9.0,-12.0,-47.0,-13.0,-6.0,1.0


In [88]:
df_mr[df_mr.iso_3166_2_code == 'FI-18'].sort_values(by = 'date',ascending=False).iloc[0:4,11].mean()

0.25

In [86]:
df_temp = df_mobility[(df_mobility.duplicated(subset = ['date','province'],keep='first')) | \
            (df_mobility.duplicated(subset = ['date','province'],keep='last'))]\
            .sort_values(by = ['province','Y-W'])
df_temp[df_temp.province == 'Uusimaa']

,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,Y-W,province
848,2020-12-28,-40.250000,-24.000000,3.500000,-56.000000,-39.750000,13.250000,2020-53,Uusimaa
1914,2020-12-28,-23.666667,-0.333333,-1.333333,-54.333333,-62.333333,16.333333,2020-53,Uusimaa
900,2021-12-27,-26.250000,0.750000,29.500000,-48.250000,-60.500000,16.000000,2021-52,Uusimaa
1098,2021-12-27,-41.333333,-18.000000,9.000000,-47.333333,-32.333333,9.666667,2021-52,Uusimaa
